# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
cities = pd.read_csv("../../Part 1 - WeatherPy/output_data/cities_data.csv")
cities = cities.drop(columns=['Unnamed: 0'])
cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Adwa,14.16,38.90,14.17,47,0,1.24,ET,1604097144
1,Naze,28.37,129.48,23.00,60,40,9.30,JP,1604097144
2,Moratuwa,6.77,79.88,27.00,88,40,1.00,LK,1604097145
3,Sydney,-33.87,151.21,19.44,88,75,6.70,AU,1604097238
4,Hobart,-42.88,147.33,15.56,76,20,2.60,AU,1604096879


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
#Configure gmaps
gmaps.configure(api_key="INSERT YOUR API KEY HERE")

In [31]:
#Set locations for gmap based on latitude and longitude
hum_locations = cities[["Lat", "Lng"]]

#Set humidity variale for heat layer
humidity = cities["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(hum_locations, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius=10)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Set up dataframe for temperatures between 25 and 20 degrees celsius,
#winds less than 10 kmh and zero cloudiness.

cities = cities[cities["Cloudiness"] <= 0]
cities = cities[cities["Wind Speed"] <= 15]
cities = cities[cities["Max Temp"] >= 20]
cities = cities[cities["Max Temp"] < 25]
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Viedma,-40.81,-63.00,23.33,21,0,0.89,AR,1604097173
74,Sakakah,29.97,40.21,20.00,13,0,2.60,SA,1604097016
125,Tiznit Province,29.58,-9.50,21.84,23,0,2.13,MA,1604097215
165,Rikitea,-23.12,-134.97,24.56,65,0,2.71,PF,1604097237
166,Canary Islands,28.00,-15.50,23.89,56,0,4.60,ES,1604097238
195,Pozo Colorado,-23.49,-58.80,21.00,52,0,3.12,PY,1604097481
217,Muhafazat Lahij,13.17,44.58,23.93,74,0,2.13,YE,1604097493
229,Dalmau,26.07,81.03,20.04,30,0,2.33,IN,1604097500
393,Pasni,25.26,63.47,23.20,31,0,6.52,PK,1604097595
421,Casablanca,33.59,-7.62,20.00,63,0,1.00,MA,1604097412


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cities

In [7]:
#Set up gmaps place finder
target_type = "hotel"
target_radius = 5000
gkey = "INSERT YOUR API KEY HERE"
target_search = "hotel"

Lat = hotel_df["Lat"]
Lng = hotel_df["Lng"]
Hotel_Name = []

for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    target_coordinates = (f"{Lat},{Lng}")

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()
    try:
        Hotel_Name.append(places_data["results"][0]["name"])
    except IndexError:
        Hotel_Name.append("N/A")
        

#Add hotels list to Hotel dataframe      
hotel_df["Hotel Name"] = Hotel_Name

In [8]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!= "N/A"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Viedma,-40.81,-63.00,23.33,21,0,0.89,AR,1604097173,Casa Crespo Hotel Boutique
74,Sakakah,29.97,40.21,20.00,13,0,2.60,SA,1604097016,Alnuzl Special Hotel
165,Rikitea,-23.12,-134.97,24.56,65,0,2.71,PF,1604097237,Pension Maro'i
166,Canary Islands,28.00,-15.50,23.89,56,0,4.60,ES,1604097238,Hotel Escuela Santa Brígida
195,Pozo Colorado,-23.49,-58.80,21.00,52,0,3.12,PY,1604097481,Hotel Pozo Colorado
393,Pasni,25.26,63.47,23.20,31,0,6.52,PK,1604097595,Juddi Hotel
421,Casablanca,33.59,-7.62,20.00,63,0,1.00,MA,1604097412,Four Seasons Hotel Casablanca
470,St. Pete Beach,27.73,-82.74,22.22,55,0,1.34,US,1604097647,The Beachcomber - St. Pete Beach


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))